In [1]:
import os
import cython
%load_ext Cython


In [2]:
%%timeit
_ = os.urandom(2000000)

4.93 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
%%cython --annotate

cdef int a = 0
cdef int i = 0
for i in range(10):
    a += i
print(a)

45


In [4]:
%%cython --annotate

from libc.stdlib cimport malloc

cdef class AES:

    cdef readonly char[256] s_box
    cdef readonly char[256] inv_s_box
    cdef readonly char[24] rcon
    cdef unsigned char *key_schedule
    cdef int nk, nr

    def __init__(self, t, key_):
        cdef int i
        cdef unsigned char *key

        self.s_box = [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
                      0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
                      0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
                      0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
                      0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
                      0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
                      0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
                      0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
                      0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
                      0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
                      0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
                      0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
                      0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
                      0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
                      0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
                      0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]

        self.inv_s_box = [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
                          0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
                          0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
                          0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
                          0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
                          0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
                          0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
                          0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
                          0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
                          0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
                          0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
                          0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
                          0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
                          0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
                          0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
                          0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d]


        self.rcon = [0x8d, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36, 0x6c, 0xd8, 0xab, 0x4d,
                           0x9a, 0x2f, 0x5e, 0xbc, 0x63, 0xc6, 0x97, 0x35, 0x6a]

        self.nr = {128:10, 192:12, 256:14}[t]
        self.nk = {128:4, 192:6, 256:8}[t]

        key = <unsigned char *> malloc(len(key_) + 1)

        for i in range(len(key_)):
            key[i] = key_[i]
        self.key_schedule = <unsigned char *> malloc((self.nr + 1)*16 + 1)
        self.key_expansion(key)

    cdef void key_expansion(self, unsigned char *key):

        cdef int i, j
        cdef unsigned char temp[4]

        for i in range(4 * self.nk):
            self.key_schedule[i] = key[i]
        for j in range(4):
            temp[j] =  key[4*self.nk - 4 + j]

        for i in range(self.nk, 4 * (self.nr + 1)):
            if not i % self.nk:
                temp[0], temp[1], temp[2], temp[3] = temp[1], temp[2], temp[3], temp[0]
                self.sub_bytes(4, temp)
                temp[0] ^= self.rcon[i // self.nk]
            elif self.nk > 6 and i % self.nk == 4:
                self.sub_bytes(4, temp)

            for j in range(4):
                self.key_schedule[4*i + j] = self.key_schedule[4*(i-self.nk) + j] ^ temp[j]
                temp[j] = self.key_schedule[4*i + j]

    cdef void add_round_key(self, unsigned char *state, int round_):
        """
        A round_ key is added to a given state
        Args:
            state: 
            round_:  
        """
        cdef int i
        for i in range(16):
            state[i] ^= self.key_schedule[round_*16 + i]



    cdef void sub_bytes(self, int n, unsigned char *state):
        """
        Substitutes each element of the state 'state' using the Rijndael S-box.
        Args:
            n: len(state)
            state: Pointer to char array
        """
        cdef int i

        for i in range(n):
            state[i] = self.s_box[state[i]]

    cdef void inv_sub_bytes(self, unsigned char *state):
        """
        Substitutes each element of the state 'state' using the Rijndael S-box.
        Args:
            state: Pointer to char array
        """
        for i in range(16):
            state[i] = self.inv_s_box[state[i]]

    @staticmethod
    cdef void shift_rows(unsigned char *state):
        """
        Shift rows of a given state
        Args:
            state: Pointer to char array
        """
        # Shift 1
        temp = state[1]
        state[1] = state[5]
        state[5] = state[9]
        state[9] = state[13]
        state[13] = temp

        # Shift 2
        temp = state[2]
        state[2] = state[10]
        state[10] = temp

        temp = state[6]
        state[6] = state[14]
        state[14] = temp

        # Shift 3
        temp = state[15]
        state[15] = state[11]
        state[11] = state[7]
        state[7] = state[3]
        state[3] = temp

    @staticmethod
    cdef void inv_shift_rows(unsigned char *state):
        """
        Shift rows of a given state
        Args:
            state: Pointer to char array
        """
        cdef unsigned char temp
        # Shift 1
        temp = state[13]
        state[13] = state[9]
        state[9] = state[5]
        state[5] = state[1]
        state[1] = temp

        # Shift 2
        temp = state[2]
        state[2] = state[10]
        state[10] = temp

        temp = state[6]
        state[6] = state[14]
        state[14] = temp

        # Shift 3
        temp = state[3]
        state[3] = state[7]
        state[7] = state[11]
        state[11] = state[15]
        state[15] = temp

    @staticmethod
    cdef void mix_columns(unsigned char *state):
        """
        Mix columns of a given state
        Args:
            state: Pointer to char array
        """
        cdef unsigned char c_2[4]
        cdef unsigned char c[4]
        cdef int i
        cdef int j

        for j in range(4):
            for i in range(4):
                c[i] = state[4*j + i]
                c_2[i] = state[4*j + i] << 1
                if c[i] & 0x80:
                    c_2[i] ^= 0x1b

            state[4*j] = c_2[0] ^ c_2[1] ^ c[1] ^ c[2] ^ c[3]
            state[4*j + 1] = c[0] ^ c_2[1] ^ c_2[2] ^ c[2] ^ c[3]
            state[4*j + 2] = c[0] ^ c[1] ^ c_2[2] ^ c_2[3] ^ c[3]
            state[4*j + 3] = c_2[0] ^ c[0] ^ c[1] ^ c[2] ^ c_2[3]

    @staticmethod
    cdef unsigned char rijndael_gf_mul(unsigned char x, unsigned char y):

        cdef unsigned char mask, res = 0
        cdef int i

        for i in range(8):
            res ^= -(y & 1) & x
            mask = -((x >> 7) & 1)
            x = (x << 1) ^(0x1b & mask)
            y >>= 1

        return res

    cdef void inv_mix_columns(self, unsigned char *state):
        """
        Inverse mix columns of a given state
        Args:
            state: 
        """
        cdef unsigned char c[4]
        cdef int i

        for j in range(4):
            for i in range(4):
                c[i] = state[4*j + i]

            state[4*j] = AES.rijndael_gf_mul(0xe, c[0]) ^ AES.rijndael_gf_mul(0xb, c[1]) ^ \
                       AES.rijndael_gf_mul(0xd, c[2]) ^ AES.rijndael_gf_mul(0x9, c[3])

            state[4*j + 1] = AES.rijndael_gf_mul(0x9, c[0]) ^ AES.rijndael_gf_mul(0xe, c[1]) ^ \
                       AES.rijndael_gf_mul(0xb, c[2]) ^ AES.rijndael_gf_mul(0xd, c[3])

            state[4*j + 2] = AES.rijndael_gf_mul(0xd, c[0]) ^ AES.rijndael_gf_mul(0x9, c[1]) ^ \
                       AES.rijndael_gf_mul(0xe, c[2]) ^ AES.rijndael_gf_mul(0xb, c[3])

            state[4*j + 3] = AES.rijndael_gf_mul(0xb, c[0]) ^ AES.rijndael_gf_mul(0xd, c[1]) ^ \
                       AES.rijndael_gf_mul(0x9, c[2]) ^ AES.rijndael_gf_mul(0xe, c[3])

    cdef void cipher_(self, unsigned char *state):
        """
        Cypher a given state
        Args:
            state: 
        """
        cdef int round_
        cdef unsigned char[16] temp

        self.add_round_key(state, 0)

        for round_ in range(1, self.nr):
            self.sub_bytes(16, state)
            AES.shift_rows(state)
            AES.mix_columns(state)
            self.add_round_key(state, round_)

        self.sub_bytes(16, state)
        AES.shift_rows(state)
        self.add_round_key(state, self.nr)

    def cipher(self, state):
        """
        Encrypts a given state
        Args:
            state (list(bytes (int))): State to encrypt. It must have at leats 16 elements.

        Returns:
            list(bytes (int))

        """
        cdef unsigned char[16] state_
        for i in range(16):
            state_[i] = state[i]

        self.cipher_(state_)
        return [state_[i] for i in range(16)]

    cdef void inv_cipher_(self, unsigned char *state):
        """
        Inverse cipher a given state
        Args:
            state: 
        """
        cdef int round_
        cdef unsigned char[16] temp

        self.add_round_key(state, self.nr)
        for round_ in range(self.nr - 1, 0, -1):
            AES.inv_shift_rows(state)
            self.inv_sub_bytes(state)
            self.add_round_key(state, round_)
            self.inv_mix_columns(state)

        AES.inv_shift_rows(state)
        self.inv_sub_bytes(state)
        self.add_round_key(state, 0)

    def inv_cipher(self, state):
        """
        Decrypts a given state
        Args:
            state (list(bytes)): Ciphered state to decrypt

        Returns:
            list(bytes (int))
        """
        cdef unsigned char[16] state_
        for i in range(16):
            state_[i] = state[i]

        self.inv_cipher_(state_)
        return [state_[i] for i in range(16)]

import os
key = os.urandom(16)
plain = os.urandom(16)
ctx = AES(None, key)
ciph = ctx.cipher(plain)
 


KeyError: 

In [4]:
%%cython --annotate
#cython: boundcheck=False
import cython

cdef unsigned char[256] s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

cdef unsigned char[256] inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

cdef unsigned char[32] r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

cdef dump_state(unsigned char[4][4] s):
    for i in range(4):
        for j in range(4):
            print(f'{s[i][j]:02x}', end=' ')
        print()

cdef sub_bytes(unsigned char[4][4] s):
    cdef int i, j
    for i in range(4):
        for j in range(4):
            s[i][j] = s_box[s[i][j]]

cdef inv_sub_bytes(unsigned char[4][4] s):
    cdef int i, j
    for i in range(4):
        for j in range(4):
            s[i][j] = inv_s_box[s[i][j]]

cdef shift_rows(unsigned char[4][4] s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

cdef inv_shift_rows(unsigned char[4][4] s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]

cdef add_round_key(unsigned char[4][4] s, unsigned char[4][4] k):
    cdef int i, j
    for i in range(4):
        for j in range(4):
            s[i][j] ^= k[i][j]

# learned from https://web.archive.org/web/20100626212235/http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
cdef unsigned char xtime(unsigned char a):
    return (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

cdef mix_single_column(unsigned char[4] a):
    # see Sec 4.1.2 in The Design of Rijndael
    cdef unsigned char t = a[0] ^ a[1] ^ a[2] ^ a[3]
    cdef unsigned char u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)

cdef mix_columns(unsigned char[4][4] s):
    cdef int i
    for i in range(4):
        mix_single_column(s[i])

cdef inv_mix_columns(unsigned char[4][4] s):
    # see Sec 4.1.3 in The Design of Rijndael
    cdef int i
    cdef unsigned char u, v
    for i in range(4):
        u = xtime(xtime(s[i][0] ^ s[i][2]))
        v = xtime(xtime(s[i][1] ^ s[i][3]))
        s[i][0] ^= u
        s[i][1] ^= v
        s[i][2] ^= u
        s[i][3] ^= v

    mix_columns(s)

# cdef expand_key(unsigned char[4][4] key_columns):
#     # TODO: add 192 and 256 keys
#     """
#     Expands and returns a list of key matrices for the given master_key.
#     """

#     cdef int n_rounds = 10
#     cdef int iteration_size = 4
#     cdef int i = 1
#     cdef int len_key_columns = 4

#     while len_key_columns < (n_rounds + 1) * 4:
#         # Copy previous word.
#         word = list(key_columns[-1])

#         # Perform schedule_core once every "row".
#         if len_key_columns % iteration_size == 0:
#             # Circular shift.
#             word.append(word.pop(0))
#             # Map to S-BOX.
#             word = [s_box[b] for b in word]
#             # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
#             word[0] ^= r_con[i]
#             i += 1
#         elif len(master_key) == 32 and len_key_columns % iteration_size == 4:
#             # Run word through S-box in the fourth iteration when using a
#             # 256-bit key.
#             word = [s_box[b] for b in word]

#         # XOR with equivalent word from previous iteration.
#         word = xor_bytes(word, key_columns[-iteration_size])
#         key_columns.append(word)

#     # Group key words in 4x4 byte matrices.
#     return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

cdef unsigned char[4][4] state = [[ 1, 2, 3, 4], [ 5, 6, 7, 8], [ 9,10,11,12], [13,14,15,16]]
cdef unsigned char[4][4] rk    = [[21,22,23,24], [25,26,27,28], [29,30,31,32], [33,34,35,36]]

sub_bytes(state)
# dump_state(state)
inv_sub_bytes(state)
# dump_state(state)
shift_rows(state)
# dump_state(state)
inv_shift_rows(state)
# dump_state(state)
add_round_key(state,rk)
# dump_state(state)
add_round_key(state,rk)
# dump_state(state)

mix_columns(state)
dump_state(state)
inv_mix_columns(state)
dump_state(state)




03 04 09 0a 
0f 08 15 1e 
0b 0c 01 02 
17 10 2d 36 
01 02 03 04 
05 06 07 08 
09 0a 0b 0c 
0d 0e 0f 10 


In [2]:
import cython
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [3]:
%%cython
from libc.string cimport memcpy

cdef unsigned char[256] s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

cdef unsigned char[256] inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

cdef unsigned char[32] r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

# cdef check_int(unsigned char[8] s):
#     # The cast interprets as big endian
#     cdef unsigned int* ui_view = <unsigned int*> s
#     print(" ".join(f'{x:02x}' for x in s))
#     ui_view[0] = ui_view[0] >> 8 | ui_view[0] << 24 
#     print(" ".join(f'{x:02x}' for x in s))
#     cdef int i
#     for i in range(4):
#         s[i] = s_box[s[i]]
#     print(" ".join(f'{x:02x}' for x in s))
#     s[0] ^= r_con[1]
#     print(" ".join(f'{x:02x}' for x in s))
#     print('!!!!!')

# cdef unsigned char[8] test = [0x6d, 0x6e, 0x6f, 0x70, 5,6,7,8]
# check_int(test)

cdef expand_key(unsigned char[16] master_key):
    cdef int n_rounds = 10
    cdef int len_key_columns = 4*11
    cdef int iteration_size = 16 // 4
    
    cdef unsigned char[4*11][4] key_columns
    cdef unsigned int* tmp
    cdef int i, j, rcon_i = 1
    
    cdef unsigned char* current_col

    memcpy(key_columns[0], master_key, 16)

    for i in range(4, len_key_columns):
        # Copy previous word 
        memcpy(key_columns[i], key_columns[i - 1], 4)
        
        if i % iteration_size == 0:
            # Circular shift.
            tmp = <unsigned int*> key_columns[i]
            tmp[0] = tmp[0] >> 8 | tmp[0] << 24

            # Map to S-BOX.
            for j in range(4):
                key_columns[i][j] = s_box[key_columns[i][j]]

            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            key_columns[i][0] ^= r_con[rcon_i]
            rcon_i += 1
        else:
            pass

        # XOR with equivalent word from previous iteration.
        tmp = <unsigned int*> key_columns[i]
        tmp[0] = tmp[0] ^ tmp[-4]

    for col in range(4*11):
        for b in range(4):
            print(f'{key_columns[col][b]:02x}', end=' ')
        print() 

cdef unsigned char[16] test2 = [0x61, 0x62, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 0x6a, 0x6b, 0x6c, 0x6d, 0x6e, 0x6f, 0x70]

expand_key(test2)



61 62 63 64 
65 66 67 68 
69 6a 6b 6c 
6d 6e 6f 70 
ff ca 32 58 
9a ac 55 30 
f3 c6 3e 5c 
9e a8 51 2c 
3f 1b 43 53 
a5 b7 16 63 
56 71 28 3f 
c8 d9 79 13 
0e ad 3e bb 
ab 1a 28 d8 
fd 6b 00 e7 
35 b2 79 f4 
31 1b 81 2d 
9a 01 a9 f5 
67 6a a9 12 
52 d8 d0 e6 
40 6b 0f 2d 
da 6a a6 d8 
bd 00 0f ca 
ef d8 df 2c 
01 f5 7e f2 
db 9f d8 2a 
66 9f d7 e0 
89 47 08 cc 
e1 c5 35 55 
3a 5a ed 7f 
5c c5 3a 9f 
d5 82 32 53 
72 e6 d8 56 
48 bc 35 29 
14 79 0f b6 
c1 fb 3d e5 
66 c1 01 2e 
2e 7d 34 07 
3a 04 3b b1 
fb ff 06 54 
46 ae 21 21 
68 d3 15 26 
52 d7 2e 97 
a9 28 28 c3 


In [16]:
%%cython
from libc.string cimport memcpy

cdef unsigned char[256] s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

cdef unsigned char[256] inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

cdef unsigned char[32] r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

# cdef check_int(unsigned char[8] s):
#     # The cast interprets as big endian
#     cdef unsigned int* ui_view = <unsigned int*> s
#     print(" ".join(f'{x:02x}' for x in s))
#     ui_view[0] = ui_view[0] >> 8 | ui_view[0] << 24 
#     print(" ".join(f'{x:02x}' for x in s))
#     cdef int i
#     for i in range(4):
#         s[i] = s_box[s[i]]
#     print(" ".join(f'{x:02x}' for x in s))
#     s[0] ^= r_con[1]
#     print(" ".join(f'{x:02x}' for x in s))
#     print('!!!!!')

# cdef unsigned char[8] test = [0x6d, 0x6e, 0x6f, 0x70, 5,6,7,8]
# check_int(test)

cdef expand_key(unsigned char[16] master_key):
    cdef int n_rounds = 10
    cdef int len_key_columns = 4*11
    cdef int iteration_size = 16 // 4
    
    cdef unsigned int[4*11] key_columns
    cdef unsigned int* tmp
    cdef int i, j, rcon_i = 1
    
    cdef unsigned char* current_col

    memcpy(key_columns, master_key, 16)

    for i in range(4, len_key_columns):
        # Copy previous word
        key_columns[i] = key_columns[i - 1]
        
        if i % iteration_size == 0:
            # Circular shift.
            key_columns[i] = key_columns[i] >> 8 | key_columns[i] << 24

            # Map to S-BOX.
            key_columns[i] = \
                s_box[key_columns[i] & 0xff]               | \
                s_box[(key_columns[i] >> 8 ) & 0xff] << 8  | \
                s_box[(key_columns[i] >> 16) & 0xff] << 16 | \
                s_box[(key_columns[i] >> 24) & 0xff] << 24

            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            key_columns[i] ^= r_con[rcon_i]
            rcon_i += 1
        else:
            # TODO
            pass

        # XOR with equivalent word from previous iteration.
        key_columns[i] ^= key_columns[i-4]



    #     # print('bla')
    #     # break


    for col in range(4*11):
        print(f'{key_columns[col]:08x}')

cdef unsigned char[16] test2 = [0x61, 0x62, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69, 0x6a, 0x6b, 0x6c, 0x6d, 0x6e, 0x6f, 0x70]

expand_key(test2)



64636261
68676665
6c6b6a69
706f6e6d
5832caff
3055ac9a
5c3ec6f3
2c51a89e
53431b3f
6316b7a5
3f287156
1379d9c8
bb3ead0e
d8281aab
e7006bfd
f479b235
2d811b31
f5a9019a
12a96a67
e6d0d852
2d0f6b40
d8a66ada
ca0f00bd
2cdfd8ef
f27ef501
2ad89fdb
e0d79f66
cc084789
5535c5e1
7fed5a3a
9f3ac55c
533282d5
56d8e672
2935bc48
b60f7914
e53dfbc1
2e01c166
07347d2e
b13b043a
5406fffb
2121ae46
2615d368
972ed752
c32828a9


In [10]:
s_box = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

inv_s_box = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes(sum(matrix, []))

def xor_bytes(a, b):
    """ Returns a new byte array with the elements xor'ed. """
    return [i^j for i, j in zip(a, b)]

def dump_state(s):
    for i in range(4):
        for j in range(4):
            print(f'{s[i][j]:02x}', end=' ')
        print()

def _expand_key(master_key):
    """
    Expands and returns a list of key matrices for the given master_key.
    """
    # Initialize round keys with raw key material.
    key_columns = bytes2matrix(master_key)
    # dump_state(key_columns)
    iteration_size = len(master_key) // 4
    n_rounds = 10
    len_key_columns = len(key_columns)

    i = 1
    while len_key_columns < (n_rounds + 1) * 4:
        # print(f'{i=}')
        # Copy previous word.
        word = list(key_columns[-1])
        print('word=' + " ".join(f'{x:02x}' for x in word))

        # Perform schedule_core once every "row".
        if len_key_columns % iteration_size == 0:
            # Circular shift.
            word.append(word.pop(0))
            print('i word=' + " ".join(f'{x:02x}' for x in word))
            # Map to S-BOX.
            word = [s_box[b] for b in word]
            print('i word=' + " ".join(f'{x:02x}' for x in word))
            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            word[0] ^= r_con[i]
            print('i word=' + " ".join(f'{x:02x}' for x in word))
            i += 1
        elif len(master_key) == 32 and len_key_columns % iteration_size == 4:
            # Run word through S-box in the fourth iteration when using a
            # 256-bit key.
            word = [s_box[b] for b in word]
            print('e word=' + " ".join(f'{x:02x}' for x in word))

        # XOR with equivalent word from previous iteration.
        word = xor_bytes(word, key_columns[-iteration_size])
        print('word=' + " ".join(f'{x:02x}' for x in word))
        key_columns.append(word)
        len_key_columns += 1

    for col in key_columns:
        print(' '.join(f'{x:02x}' for x in col))

    # Group key words in 4x4 byte matrices.
    return [key_columns[4*i : 4*(i+1)] for i in range(len_key_columns // 4)]

key = b'abcdefghijklmnop'
print('[' + " ".join(f'0x{x:02x}' for x in key) + ']')
rks = _expand_key(key)
for rk in rks:
    print(matrix2bytes(rk).hex(), matrix2bytes(rk))


[0x61 0x62 0x63 0x64 0x65 0x66 0x67 0x68 0x69 0x6a 0x6b 0x6c 0x6d 0x6e 0x6f 0x70]
word=6d 6e 6f 70
i word=6e 6f 70 6d
i word=9f a8 51 3c
i word=9e a8 51 3c
word=ff ca 32 58
word=ff ca 32 58
word=9a ac 55 30
word=9a ac 55 30
word=f3 c6 3e 5c
word=f3 c6 3e 5c
word=9e a8 51 2c
word=9e a8 51 2c
i word=a8 51 2c 9e
i word=c2 d1 71 0b
i word=c0 d1 71 0b
word=3f 1b 43 53
word=3f 1b 43 53
word=a5 b7 16 63
word=a5 b7 16 63
word=56 71 28 3f
word=56 71 28 3f
word=c8 d9 79 13
word=c8 d9 79 13
i word=d9 79 13 c8
i word=35 b6 7d e8
i word=31 b6 7d e8
word=0e ad 3e bb
word=0e ad 3e bb
word=ab 1a 28 d8
word=ab 1a 28 d8
word=fd 6b 00 e7
word=fd 6b 00 e7
word=35 b2 79 f4
word=35 b2 79 f4
i word=b2 79 f4 35
i word=37 b6 bf 96
i word=3f b6 bf 96
word=31 1b 81 2d
word=31 1b 81 2d
word=9a 01 a9 f5
word=9a 01 a9 f5
word=67 6a a9 12
word=67 6a a9 12
word=52 d8 d0 e6
word=52 d8 d0 e6
i word=d8 d0 e6 52
i word=61 70 8e 00
i word=71 70 8e 00
word=40 6b 0f 2d
word=40 6b 0f 2d
word=da 6a a6 d8
word=da 6a a6 d8
word

In [7]:














r_con = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes(sum(matrix, []))

def xor_bytes(a, b):
    """ Returns a new byte array with the elements xor'ed. """
    return bytes(i^j for i, j in zip(a, b))


class AESSemi:
    """
    Class for AES-128 encryption with CBC mode and PKCS#7.

    This is a raw implementation of AES, without key stretching or IV
    management. Unless you need that, please use `encrypt` and `decrypt`.
    """
    rounds_by_key_size = {16: 10, 24: 12, 32: 14}
    def __init__(self, master_key, print_states=False):
        """
        Initializes the object with a given key.
        """
        assert len(master_key) in AESSemi.rounds_by_key_size
        self.n_rounds = AESSemi.rounds_by_key_size[len(master_key)]
        self._key_matrices = self._expand_key(master_key)
        self.print_states = print_states

    def _expand_key(self, master_key):
        """
        Expands and returns a list of key matrices for the given master_key.
        """
        # Initialize round keys with raw key material.
        key_columns = bytes2matrix(master_key)
        iteration_size = len(master_key) // 4

        i = 1
        while len(key_columns) < (self.n_rounds + 1) * 4:
            # Copy previous word.
            word = list(key_columns[-1])

            # Perform schedule_core once every "row".
            if len(key_columns) % iteration_size == 0:
                # Circular shift.
                word.append(word.pop(0))
                # Map to S-BOX.
                word = [s_box[b] for b in word]
                # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
                word[0] ^= r_con[i]
                i += 1
            elif len(master_key) == 32 and len(key_columns) % iteration_size == 4:
                # Run word through S-box in the fourth iteration when using a
                # 256-bit key.
                word = [s_box[b] for b in word]

            # XOR with equivalent word from previous iteration.
            word = xor_bytes(word, key_columns[-iteration_size])
            key_columns.append(word)

        # Group key words in 4x4 byte matrices.
        return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

    def partial_encrypt_block(self, bias_round, bias_operation, bias_state):
        """
        Perform partial encryption to finish the biased ronud to get the corresponding output.

        - bias_round: Round to start from, given bias_state.
        - bias_operation: Operation within bias_round to start from, given bias_state.
        - bias_state: biased state.
        """
        assert len(bias_state) == 16
        assert 0 <= bias_round <= self.n_rounds
        assert bias_operation in ['add_round_key', 'sub_bytes', 'shift_rows', 'mix_columns']

        plain_state = bytes2matrix(bias_state)
        do_next = False

        start_round = 0
        if bias_round == 0:
            assert bias_operation == 'add_round_key'
            if self.print_states: print(f'{0:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
            add_round_key(plain_state, self._key_matrices[0])
            do_next = True
            start_round = 1
        elif 1 <= bias_round < self.n_rounds:
            start_round = bias_round
        else: 
            start_round = bias_round

        for i in range(start_round, self.n_rounds):
            if do_next or bias_operation == 'sub_bytes':
                if self.print_states: print(f'{i:3d} {"sub_bytes":15s} {matrix2bytes(plain_state).hex()}')
                sub_bytes(plain_state)
                do_next = True
            if do_next or bias_operation == 'shift_rows':
                if self.print_states: print(f'{i:3d} {"shift_rows":15s} {matrix2bytes(plain_state).hex()}')
                shift_rows(plain_state)
                do_next = True
            if do_next or bias_operation == 'mix_columns':
                if self.print_states: print(f'{i:3d} {"mix_columns":15s} {matrix2bytes(plain_state).hex()}')
                mix_columns(plain_state)
                do_next = True
            if do_next or bias_operation == 'add_round_key':
                if self.print_states: print(f'{i:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
                add_round_key(plain_state, self._key_matrices[i])
                do_next = True

        if do_next or start_round == self.n_rounds:
            if do_next or bias_operation == 'sub_bytes':
                if self.print_states: print(f'{self.n_rounds:3d} {"sub_bytes":15s} {matrix2bytes(plain_state).hex()}')
                sub_bytes(plain_state)
                do_next = True
            if do_next or bias_operation == 'shift_rows':
                if self.print_states: print(f'{self.n_rounds:3d} {"shift_rows":15s} {matrix2bytes(plain_state).hex()}')
                shift_rows(plain_state)
                do_next = True
            if do_next or bias_operation == 'add_round_key':
                if self.print_states: print(f'{self.n_rounds:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
                add_round_key(plain_state, self._key_matrices[-1])
                do_next = True

        return matrix2bytes(plain_state)
    
    def encrypt_block(self, plaintext):
        """
        Encrypts a single block of 16 byte long plaintext.
        """
        assert len(plaintext) == 16

        plain_state = bytes2matrix(plaintext)

        if self.print_states: print(f'{0:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
        add_round_key(plain_state, self._key_matrices[0])

        for i in range(1, self.n_rounds):
            if self.print_states: print(f'{i:3d} {"sub_bytes":15s} {matrix2bytes(plain_state).hex()}')
            sub_bytes(plain_state)
            if self.print_states: print(f'{i:3d} {"shift_rows":15s} {matrix2bytes(plain_state).hex()}')
            shift_rows(plain_state)
            if self.print_states: print(f'{i:3d} {"mix_columns":15s} {matrix2bytes(plain_state).hex()}')
            mix_columns(plain_state)
            if self.print_states: print(f'{i:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
            add_round_key(plain_state, self._key_matrices[i])

        if self.print_states: print(f'{self.n_rounds:3d} {"sub_bytes":15s} {matrix2bytes(plain_state).hex()}')
        sub_bytes(plain_state)
        if self.print_states: print(f'{self.n_rounds:3d} {"shift_rows":15s} {matrix2bytes(plain_state).hex()}')
        shift_rows(plain_state)
        if self.print_states: print(f'{self.n_rounds:3d} {"add_round_key":15s} {matrix2bytes(plain_state).hex()}')
        add_round_key(plain_state, self._key_matrices[-1])

        return matrix2bytes(plain_state)
    
    def decrypt_block(self, ciphertext):
        """
        Decrypts a single block of 16 byte long ciphertext.
        """
        assert len(ciphertext) == 16

        cipher_state = bytes2matrix(ciphertext)

        add_round_key(cipher_state, self._key_matrices[-1])
        if self.print_states: print(f'{self.n_rounds:3d} {"add_round_key":15s} {matrix2bytes(cipher_state).hex()}')
        inv_shift_rows(cipher_state)
        if self.print_states: print(f'{self.n_rounds:3d} {"shift_rows":15s} {matrix2bytes(cipher_state).hex()}')
        inv_sub_bytes(cipher_state)
        if self.print_states: print(f'{self.n_rounds:3d} {"sub_bytes":15s} {matrix2bytes(cipher_state).hex()}')

        for i in range(self.n_rounds - 1, 0, -1):
            add_round_key(cipher_state, self._key_matrices[i])
            if self.print_states: print(f'{i:3d} {"sub_bytes":15s} {matrix2bytes(cipher_state).hex()}')
            inv_mix_columns(cipher_state)
            if self.print_states: print(f'{i:3d} {"shift_rows":15s} {matrix2bytes(cipher_state).hex()}')
            inv_shift_rows(cipher_state)
            if self.print_states: print(f'{i:3d} {"mix_columns":15s} {matrix2bytes(cipher_state).hex()}')
            inv_sub_bytes(cipher_state)
            if self.print_states: print(f'{i:3d} {"add_round_key":15s} {matrix2bytes(cipher_state).hex()}')

        add_round_key(cipher_state, self._key_matrices[0])
        if self.print_states: print(f'{0:3d} {"add_round_key":15s} {matrix2bytes(cipher_state).hex()}')
    
        return matrix2bytes(cipher_state)
    
